In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Define constants
im_size = 224
batch_size = 16
epochs = 10
dataset_path = '/content/drive/MyDrive/Plant_Directory/Dataset'

In [8]:
# Load and preprocess data
class_names = ['peach_bacterial', 'peach_healthy']
print('Class names:', class_names)
images = []
labels = []

for i, class_name in enumerate(class_names):
    data_path = os.path.join(dataset_path, class_name)
    filenames = os.listdir(data_path)

    for filename in filenames:
        img = cv2.imread(os.path.join(data_path, filename))
        img = cv2.resize(img, (im_size, im_size))
        images.append(img)
        labels.append(i)

images = np.array(images)
labels = np.array(labels)
labels = labels.reshape(-1, 1)
images = images.astype('float32') / 255.0


Class names: ['peach_bacterial', 'peach_healthy']


In [9]:
# Split data into train and test sets
train_x, test_x, train_y, test_y = train_test_split(images, labels, test_size=0.2, random_state=200)

In [10]:
# Create ResNet-50 model
base_model = ResNet50(input_shape=(im_size, im_size, 3), include_top=False, weights='imagenet')
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)

94765736/94765736 [==============================] - 0s 0us/step


In [11]:
# Compile the model
model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [12]:
# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('binary_classifier.h5', save_best_only=True)

In [13]:
# Train the model
history = model.fit(train_x, train_y, validation_data=(test_x, test_y), batch_size=batch_size, epochs=epochs, callbacks=[early_stopping, model_checkpoint])

Epoch 1/20
191/191 [==============================] - 110s 289ms/step - loss: 0.1340 - accuracy: 0.9639 - val_loss: 0.6933 - val_accuracy: 0.4541
Epoch 2/20
191/191 [==============================] - 35s 184ms/step - loss: 0.0735 - accuracy: 0.9790 - val_loss: 2.4909 - val_accuracy: 0.5459
Epoch 3/20
191/191 [==============================] - 36s 189ms/step - loss: 0.0383 - accuracy: 0.9849 - val_loss: 0.9020 - val_accuracy: 0.5787
Epoch 4/20
191/191 [==============================] - 36s 187ms/step - loss: 0.0360 - accuracy: 0.9905 - val_loss: 0.8769 - val_accuracy: 0.5472
Epoch 5/20
191/191 [==============================] - 57s 298ms/step - loss: 0.0092 - accuracy: 0.9970 - val_loss: 0.5872 - val_accuracy: 0.7310
Epoch 6/20
191/191 [==============================] - 36s 190ms/step - loss: 0.0122 - accuracy: 0.9970 - val_loss: 2.3368 - val_accuracy: 0.6063
Epoch 7/20
191/191 [==============================] - 62s 325ms/step - loss: 0.1426 - accuracy: 0.9642 - val_loss: 0.2439 - val_a

In [14]:
# Evaluate the model on test data
loss, accuracy = model.evaluate(test_x, test_y, batch_size=batch_size)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

48/48 [==============================] - 3s 54ms/step - loss: 0.0135 - accuracy: 1.0000
Test Loss: 0.013546171598136425
Test Accuracy: 1.0


In [15]:
# Make predictions on new data
def preprocess_image(img):
    img = cv2.resize(img, (im_size, im_size))
    img = img.astype('float32') / 255.0
    img = np.expand_dims(img, axis=0)
    return img

In [18]:
# Example usage to predict on a new image
new_image_path = '/content/drive/MyDrive/Plant_Directory/Dataset/peach_bacterial/00130039-8425-42e9-9dd9-15aead7271ff___Rut._Bact.S 3421.JPG'
new_image = cv2.imread(new_image_path)
preprocessed_img = preprocess_image(new_image)
prediction = model.predict(preprocessed_img)

class_names = ['Bacterial Spot', 'Healthy']
predicted_class = class_names[int(prediction[0][0] > 0.5)]

print(f"Predicted Class: {predicted_class}")


1/1 [==============================] - 0s 30ms/step
Predicted Class: Bacterial Spot


In [19]:
model.save('/content/drive/MyDrive/binary_classifier_model.h5')